# SARS-CoV-2 Exploration

## Acknowledgement 

This tutorial is based on work done by:
- Chris Rands [biopython-coronavirus](https://github.com/chris-rands/biopython-coronavirus)
- Harshit Tyagi [COVID-19 Dashboard](https://github.com/dswh/voila-covid-19-dashboard/blob/master/notebooks/covid_19_dashboard.ipynb)

In [ ]:
# install a few missing libraries this will take a few minutes

!pip3 install plotly==4.9.0 seaborn==0.10.0 folium==0.11.0 

Since we have just installed some new libraries you may need to restart Python - Go to the **Kernel** menu and select **Restart**

In [ ]:
# load needed libraries 

import os
import Bio as Bio 
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import plotly
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

## Looking at SARS-CoV-2 Sequence

We can dowload the SARS-CoV-2 Reference Genome from NCBI [Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2?report=fasta)

In [ ]:

# get text genbank record for SARS-CoV-2
Entrez.email = "Your.Name.Here@example.org" # we have to provide NCBI an email - you 
handle = Entrez.efetch(db="nucleotide", id="NC_045512", rettype="gb", retmode="text")
print(handle.read())


Get the sequence in a format that Biopython can read

In [ ]:
filename = "NC_045512.fasta"
Entrez.email = "A.N.Other@example.com"
if not os.path.isfile(filename):
    # Downloading...
    net_handle = Entrez.efetch(
        db="nucleotide", id="NC_045512", rettype="fasta", retmode="text"
    )
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()
    

print("Parsing...")
sars_cov2_refseq = SeqIO.read(filename, "fasta")
print(record)

In [ ]:
!cat NC_045512.fasta

We now have a variable caled seq_record which has certian properties

In [ ]:
print(sars_cov2_refseq)

In [ ]:
sars_cov2_refseq.id # show the ID of the sequence 

In [ ]:
str(sars_cov2_refseq.seq) # show the entire sequence

In [ ]:
print("Sequence length (bp)", len(sars_cov2_refseq)) # get the nucleotide length

In [ ]:
print("GC content (%)", GC(sars_cov2_refseq.seq))

Let's download some related viruses to make some comparisons 



In [ ]:
#Middle East Respiratory Syndrome-related coronavirus (MERS-CoV)
#Severe Acute Respiratory Syndrome-related coronavirus (SARS-Cov)


virus_dict = {"NC_038294":"NC_038294.fasta", #MERS Betacoronavirus England 1, complete genome
           "NC_004718.3":"NC_004718.3.fasta" #SARS coronavirus Tor2, complete genome
          }


for virus in virus_dict.keys():
    filename = virus_dict[virus]
    Entrez.email = "A.N.Other@example.com"
    if not os.path.isfile(filename):
        # Downloading...
        net_handle = Entrez.efetch(
            db="nucleotide", id=virus, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        


Let's see the files we have dowloaded

In [ ]:
!ls

let's load all sequences into one file so we can make some comparisons

In [ ]:
!cat NC_045512.fasta NC_038294.fasta NC_004718.3.fasta > viruses.fasta

We now have all 3 viral genomes in a single file

In [ ]:
!cat viruses.fasta

We can dowload and use a software called MUSCLE to make the alignment

In [ ]:
# Download and install MUSCLE 

# Linux 
!wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz && tar -xvf muscle3.8.31_i86linux64.tar.gz

Now we can align the viruses (this will take a few minutes). Check out the NCBI SARS-CoV-2 [sequence tree](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/precomptree) while you are waiting

In [ ]:
!./muscle3.8.31_i86linux64 -in viruses.fasta -out alignment.txt -clw

We can view the alignment here (first 10 lines of file), but it may be better to use the Jupyter browser to open the file in a new tab. 

In [ ]:
!head alignment.txt

Let's view a phylogenetic tree of this result. First here is the alignment in Biopython format

In [ ]:
from Bio import AlignIO
align = AlignIO.read("alignment.txt", "clustal")
print(align)

Make the tree with muscle...

In [ ]:
!./muscle3.8.31_i86linux64  -maketree -in viruses.fasta -out viruses.phy

And visualize with BioPython. (Tip, open the viruses.fasta file and shorten the name to make the naming more clear)

In [ ]:
from Bio import Phylo
tree = Phylo.read("viruses.phy", "newick")
Phylo.draw_ascii(tree)

# Exploring epidemiological data

John Hopkins University aggregates and organizes [COVID-19 data](https://github.com/CSSEGISandData) we can easily use. We'll import it below

In [ ]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

We can build a table that shows us each of these datasets

In [ ]:
confirmed_df.head()

In [ ]:
recovered_df.head()

In [ ]:
death_df.head()

In [ ]:
country_df.head()

There are a series of steps we can do to clean the data (changing column names, and reorganizing the data

In [ ]:
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

country_df.head()

In [ ]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

Let's look at the total stats

In [ ]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

We can summarise these data in a widget to make them interactive (and pretty). Here is. the top 10

In [ ]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

We will sort the countries for the next visualization 

In [ ]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [ ]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

In [ ]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map